# 📘 Word → CSV → YAML → 文档生成 Pipeline Demo
本 Notebook 演示如何将 **非结构化的 Word 文档（AT 命令集）** 转化为结构化数据，最终生成 Markdown 文档。

流程：Word → CSV → YAML → Markdown/PDF

In [ ]:
# 安装依赖（首次运行时需要）
!pip install python-docx pandas pyyaml jinja2

## Step 1. 从 Word 提取表格 → CSV
假设 Word 文档里有一张表格：指令 | 描述 | 示例响应。
我们用 python-docx 提取并存成 CSV。

In [ ]:
import pandas as pd
from docx import Document

# 示例：我们假设有个 word 文件 demo.docx
# 实际运行时换成你的 Word 文档路径
doc = Document()
table = doc.add_table(rows=3, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text, hdr_cells[1].text, hdr_cells[2].text = '指令', '描述', '示例响应'
table.rows[1].cells[0].text, table.rows[1].cells[1].text, table.rows[1].cells[2].text = 'AT+CSQ', '查询信号强度', '+CSQ: 25,0'
table.rows[2].cells[0].text, table.rows[2].cells[1].text, table.rows[2].cells[2].text = 'AT+CGATT', '网络附着', '+CGATT: 1'

demo_path = "demo.docx"
doc.save(demo_path)

# 重新读取并提取表格
doc = Document(demo_path)
data = []
for table in doc.tables:
    keys = None
    for i, row in enumerate(table.rows):
        text = [cell.text.strip() for cell in row.cells]
        if i == 0:
            keys = text
        else:
            data.append(dict(zip(keys, text)))

df = pd.DataFrame(data)
csv_file = "commands.csv"
df.to_csv(csv_file, index=False, encoding="utf-8")
df

## Step 2. 从 CSV 转换为 YAML
用 Pandas 读取 CSV，再输出成结构化 YAML。

In [ ]:
import yaml

df = pd.read_csv(csv_file)
records = df.to_dict(orient="records")

yaml_data = {"product": "N706B", "version": "V1.0", "commands": records}
yaml_file = "commands.yaml"
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(yaml_data, f, allow_unicode=True)

print(open(yaml_file, encoding="utf-8").read())

## Step 3. 用 Jinja2 模板生成 Markdown
我们利用结构化的 YAML 数据，统一渲染成 Markdown 文档。

In [ ]:
from jinja2 import Template

template_str = """
# {{ product }} 模块 AT 命令手册 (版本 {{ version }})

| 指令   | 描述   | 示例响应 |
|--------|--------|----------|
{% for c in commands %}| {{ c['指令'] }} | {{ c['描述'] }} | {{ c['示例响应'] }} |
{% endfor %}
"""
template = Template(template_str)

output_md = template.render(**yaml_data)
md_file = "AT_manual.md"
with open(md_file, "w", encoding="utf-8") as f:
    f.write(output_md)
print(output_md)

## Step 4. 下一步拓展
- 现在你已经能跑通 Word → CSV → YAML → Markdown。
- 可以接 Pandoc：
  ```bash
  pandoc AT_manual.md -o AT_manual.pdf
  ```
- 或者接 Sphinx / DITA-OT 做大规模文档管理。
- 更进一步：在 CI/CD 流水线里，把 Word 自动化转换为结构化数据 → 渲染 → 发布。